In [1]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append("..")

from analysis.utils import monetary_string_to_numeric, clean_player_name
from data_sources import PyBaseball, MLBStatsAPI, Salary
from analysis.batter_data_structure import KEEP_RENAME_MAP, ROLLING_COLS

from dotenv import load_dotenv
load_dotenv()

py_baseball = PyBaseball()
mlb_api = MLBStatsAPI()

payroll_source_paths = {
    "historical": os.getenv("MLB_PAYROLLS"),
    "recent": os.getenv("MLB_PAYROLLS_2025")
}

salary_source_paths = {
    "historical": os.getenv("MLB_PLAYER_SALARY_DATA")
}

salary = Salary(payroll_source_paths=payroll_source_paths, salary_source_paths=salary_source_paths)
payrolls = salary.payroll()

# Contains matching keys between data sources
# May be helpful down the road
# chadwick = py_baseball.player_search.chadwick()

import logging
logging.basicConfig(level=logging.WARNING, force=True)  # force=True resets handlers in Jupyter (Py3.8+)

for name in ("urllib3", "urllib3.connectionpool", "requests"):
    logging.getLogger(name).setLevel(logging.ERROR)
    logging.getLogger(name).propagate = False

### Utils and Consts

In [2]:
from pathlib import Path

FREE_AGENCY_2026_PATH = Path(os.getenv("MLB_FREE_AGENTS_2026"))
ROLLING_PERIOD = 3

In [3]:
def _filter_and_rename(df: pd.DataFrame, rename_map: dict):
    df = df[list(rename_map.keys())].rename(columns=rename_map)
    return df


def _reformat_statcast_name(name: str):
    name_split = name.split(", ")
    return f"{name_split[-1]} {name_split[0]}"


def _fill_missing_salary_values(df: pd.DataFrame, league_min: int):
    """Handle's free agent 
    """
    if "Value" in df.columns:
        df = df.rename({"Value": "value"}, axis=1)

    df["AAV"] = df["AAV"].fillna(league_min)
    df["value"] = df["value"].fillna(league_min)

    return df

def _aggregate_over_season_bref(bref_data: pd.DataFrame) -> pd.DataFrame:
    return (
        bref_data
        .groupby(["season", "mlb_id"])
        .agg({
            "player_name": "first",
            "team": lambda x: "/".join(x.unique()),   # teams played for that season
            "salary": "sum",
            "bWAR": "sum",
        })
        .reset_index()
    )

### Batters

Standard compile standard batter stats from fangraphs

In [4]:
def standard_batter_stats_data_preprocessing(season: int, batter_stats_collection: dict):
    batter_stats = py_baseball.batter.stats(start_season=season)
    batter_stats = _filter_and_rename(batter_stats, KEEP_RENAME_MAP["stats"])
    # Used to obtain the player salaries
    batter_bwar = py_baseball.batter.bref_war(season)
    batter_bwar = _filter_and_rename(batter_bwar, KEEP_RENAME_MAP["bref_war"])
    batter_bwar["player_name"] = batter_bwar["player_name"].apply(clean_player_name)
    batter_bwar = _aggregate_over_season_bref(batter_bwar)

    batter_stats = (
        batter_stats
        .merge(
            batter_bwar, 
            how="left", 
            on=["player_name", "team", "season"] 
            )
    )
    
    # For now assume that na values are league minimum
    league_minimum = salary.league_minimum_salaries(season)
    batter_stats["salary"] = batter_stats["salary"].fillna(np.float64(league_minimum))
    
    return batter_stats_collection | {season: batter_stats}

start_season, end_season = 2009, 2025

batter_stats = {}
for season in range(start_season, end_season + 1):
    batter_stats = standard_batter_stats_data_preprocessing(season, batter_stats)

Compile statcast data.

Data is made up of:
* Statcast expected stats: e.g. expected batting average
* Statcast percentile rankings: e.g. sprint speed percentile

In [5]:
def standard_batter_statcast_preprocessing(season: int, batter_statcast_collection: dict):
    batter_statcast_expected = py_baseball.batter.statcast_expected_stats(season)
    batter_statcast_expected = _filter_and_rename(batter_statcast_expected, KEEP_RENAME_MAP["statcast_exp"])

    batter_statcast_percentile = py_baseball.batter.statcast_percentile_ranks(season)
    batter_statcast_percentile = _filter_and_rename(batter_statcast_percentile, KEEP_RENAME_MAP["statcast_pct"])

    statcast = batter_statcast_expected.merge(batter_statcast_percentile, how="left", on=["statcast_id"])

    statcast = statcast.dropna(subset="player_name")

    statcast["player_name"] = statcast["player_name"].apply(lambda player_name: _reformat_statcast_name(player_name))
    statcast["season"] = season
    
    statcast = statcast.reset_index(drop=True)
    return batter_statcast_collection | {season: statcast}


start_season, end_season = 2015, 2025

batter_statcast = {}
for season in range(start_season, end_season + 1):
    batter_statcast = standard_batter_statcast_preprocessing(season, batter_statcast)

Compile free agents data and list sourced from spotrac.com.

Contains basic player information about historical free agents such as contract details, position, team signed, etc.

Data does not contain a matching key to the other data sources so this frame has to be merged on player name, team and season.

The season column attached to these DataFrames corresponds to the year prior to the given free agency year. This allows the free agent contract data to be matched to the statistics of the season the player just played in. 



In [6]:
def preprocess_standard_free_agent_data(
    season: int, 
    free_agent_collection: dict, 
    root_dir=os.getenv("MLB_FREE_AGENTS"),
    existing_df: pd.DataFrame | None = None,
    extra_cols_to_keep: dict = None
):
    if extra_cols_to_keep is None:
        extra_cols_to_keep = {}
        
    free_agents_path = os.path.join(root_dir, f"{season} MLB Free Agents.csv")

    if existing_df is not None:
        free_agents = existing_df

    else:
        free_agents = pd.read_csv(free_agents_path)
    # Clean up column names
    cols = [c.split(" ")[0] for c in free_agents.columns]
    free_agents.columns = cols

    free_agents = _filter_and_rename(free_agents, KEEP_RENAME_MAP["free_agents"] | extra_cols_to_keep)

    # Cast money columns to numeric values
    free_agents["value"] = free_agents["value"].apply(lambda val: monetary_string_to_numeric(val))
    free_agents["AAV"] = free_agents["AAV"].apply(lambda val: monetary_string_to_numeric(val))
    # Clean QO off of qualifying offer players
    free_agents["player_name"] = free_agents["player_name"].apply(lambda name: name.replace("QO", "").strip())

    # Set the season back a year for merging
    free_agents["season"] = season - 1

    free_agents = _fill_missing_salary_values(free_agents, salary.league_minimum_salaries(season))

    return free_agent_collection | {season: free_agents}

start_season, end_season = 2012, 2026

free_agents = {}
for season in range(start_season, end_season + 1):
    free_agents = preprocess_standard_free_agent_data(season, free_agents)

### Concatenate all batter data from fangraphs and statcast, and calculate rolling averages.

In [7]:
def get_free_agent_player_names(dfs: list[pd.DataFrame]):
    seen_players = []
    for df in dfs:
        seen_players += list(df["player_name"].unique())

    return set(seen_players)

free_agent_player_names = get_free_agent_player_names(list(free_agents.values()))

In [8]:
# Combine all batter stats
all_batter_stats = pd.concat(
    list(batter_stats.values()),
    ignore_index=True
)

# Filter to free-agent batters only
free_agent_batter_stats = (
    all_batter_stats[
        all_batter_stats["player_name"].isin(free_agent_player_names)
    ]
    .reset_index(drop=True)
)

# Add next_team using a grouped shift
free_agent_batter_stats["next_team"] = (
    free_agent_batter_stats
    .groupby("fg_id")["team"]
    .shift(-1)
)

# Combine all Statcast stats
all_statcast_stats = pd.concat(
    list(batter_statcast.values()),
    ignore_index=True
)

# Columns in Statcast but not in FA stats (plus season)
statcast_cols = (
    [col for col in all_statcast_stats.columns
     if col not in free_agent_batter_stats.columns]
    + ["season"]
)

# Merge free agent batter stats with statcast stats
free_agent_batter_stats = free_agent_batter_stats.merge(
    all_statcast_stats[statcast_cols],
    how="left",
    left_on=["mlb_id", "season"],
    right_on=["statcast_id", "season"]
)

# Combine all free agents
all_free_agents = pd.concat(
    list(free_agents.values()),
    ignore_index=True
)

Compute rolling statistics for numerical data.

In [9]:
def calculate_rolling_statistics(
        df: pd.DataFrame, 
        groupby_col="fg_id", 
        rolling_period=ROLLING_PERIOD, 
        rolling_cols=ROLLING_COLS
    ):
    """Compute rolling statistics"""
    for col in rolling_cols:
        df[f"{col}_{rolling_period}yr_rolling"] = (
            df
            .groupby(groupby_col)[col]
            .rolling(window=rolling_period, min_periods=None)
            .mean()
            .reset_index(level=0, drop=True)
        )
    return df

free_agent_batter_stats = calculate_rolling_statistics(free_agent_batter_stats)


Merge all of the free agent information together finally into a single free_agency data frame.

* free_agent_batter_stats - (batter_stats + statcast_stats)
* all_free_agents - spotrac data

In [10]:
# Merge the player stats for just the free agent players
free_agency = (
    free_agent_batter_stats
    .merge(
        all_free_agents, 
        how="left", 
        left_on=["player_name", "season", "next_team"], 
        right_on=["player_name", "season", "fa_team"]
        )
    .reset_index(drop=True)
)

# Drops the non free agency years for the players
free_agency = free_agency.dropna(subset="AAV").reset_index(drop=True)

Separate pre-statcast data and post-statcast free agent statistics.

In [13]:
# Columns to ignore for pre statcast players
pre_statcast_drop_cols = [
    col for col in free_agency.columns 
    if col.replace("_3yr_rolling", "") in statcast_cols
]

pre_statcast_free_agency = free_agency[free_agency["season"] < 2015].drop(pre_statcast_drop_cols, axis=1)
post_statcast_free_agency = free_agency[free_agency["season"] >= 2015]

Build and apply the XGBRegressor Model

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

from collections import defaultdict

exclude_cols = ["player_name", "fg_id", "fa_team", "contract_years", "next_team", "value", "AAV"]
categorical_cols = ["team", "pos"] 
numeric_cols = [
    col for col in post_statcast_free_agency.columns 
    if col not in exclude_cols 
    and col not in categorical_cols
]

def build_pre_transformer(numeric_cols, categorical_cols):
    return ColumnTransformer([
        # 1. Base numeric: impute + scale
        (
            "numeric",
            Pipeline([
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler()),
            ]),
            numeric_cols,
        ),

        # 2. Base categorical: one-hot
        (
            "base_cat",
            OneHotEncoder(handle_unknown="ignore"),
            categorical_cols,
        )
    ])

results = defaultdict(dict)
models = defaultdict(dict)

targets = ["AAV", "value", "contract_years"]

X = post_statcast_free_agency.drop(exclude_cols, axis=1)

for target in targets:
    pre = build_pre_transformer(numeric_cols, categorical_cols)
    
    model = Pipeline([
        ("pre", pre),
        ("est", XGBRegressor(n_estimators=800, max_depth=6))
    ])
    
    y = post_statcast_free_agency[target]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Save model
    models[target] = model

    # Save results
    results[target]["mae"] = mae
    results[target]["rmse"] = rmse
    results[target]["r2"] = r2

    print(f"{target}:")
    print(f"MAE:  {mae:,.3f}")
    print(f"RMSE: {rmse:,.3f}")
    print(f"R²:   {r2:.3f}")
    print("-" * 20)

c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn

AAV:
MAE:  3,687,799.345
RMSE: 6,627,832.001
R²:   0.642
--------------------


c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


value:
MAE:  29,193,548.296
RMSE: 88,665,992.139
R²:   0.420
--------------------
contract_years:
MAE:  0.973
RMSE: 1.638
R²:   0.437
--------------------


c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [16]:
free_agent_batter_stats_2026 = free_agent_batter_stats[free_agent_batter_stats["season"] == 2025]
all_free_agents_2026 = all_free_agents[all_free_agents["season"] == 2025]

free_agency_2026 = all_free_agents_2026.merge(
    free_agent_batter_stats_2026,
    how="left",
    left_on=["player_name", "season"],
    right_on=["player_name", "season"]
)

free_agency_2026 = free_agency_2026.dropna(subset="fg_id").reset_index(drop=True)

Test the model on the current free agent class. Predict the AAV, contract length, and total value.

In [17]:
X_test = free_agency_2026.drop(exclude_cols, axis=1)

# Predict on the targets
for target in targets:
    # Retrieve the model
    model = models[target]
    y_pred = model.predict(X_test)

    free_agency_2026[f"predicted_{target}"] = y_pred

free_agency_predicted_values_2026 = pd.read_csv(FREE_AGENCY_2026_PATH)
free_agency_predicted_values_2026 = free_agency_predicted_values_2026[["player_name", "market_value_AAV"]]

free_agency_2026_display = free_agency_2026[
    [
     "season", 
     "player_name", 
     "age", 
     "pos", 
     "predicted_AAV", 
     "predicted_value",
     "predicted_contract_years"
    ]
]

free_agency_2026_display = free_agency_2026_display.merge(
    free_agency_predicted_values_2026, 
    how="left", 
    on="player_name"
)

free_agency_2026_display["excess_value_predicted"] = (
    (free_agency_2026_display["predicted_AAV"] - free_agency_2026_display["market_value_AAV"])
    .apply(lambda x: x > 0)
)

for col in ["predicted_AAV", "market_value_AAV", "predicted_value"]:
    free_agency_2026_display[col] = (
        free_agency_2026_display[col]
        .apply(lambda x: f"{x:,.0f}")
    )

pd.set_option('display.max_rows', None)

free_agency_2026_display

c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['bat_speed_3yr_rolling' 'squared_up_rate_3yr_rolling'
 'swing_length_3yr_rolling']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn

,season,player_name,age,pos,predicted_AAV,predicted_value,predicted_contract_years,market_value_AAV,excess_value_predicted
0,2025,Trent Grisham,28.0,CF,"12,045,639","80,584,392",3.358333,"12,052,666",False
1,2025,Bo Bichette,27.0,SS,"19,597,496","71,838,592",3.284434,"23,303,341",False
2,2025,Gleyber Torres,28.0,2B,"13,925,476","47,047,564",2.580320,"13,971,496",False
3,2025,Kyle Tucker,28.0,RF,"27,272,796","166,024,368",4.716524,"40,179,471",False
4,2025,Kyle Schwarber,32.0,LF,"27,508,976","149,007,456",4.060745,"24,902,992",True
5,2025,Carlos Santana,39.0,1B,"5,677,203","82,173",1.027327,"6,048,900",False
6,2025,Tyler Alexander,30.0,RP,"2,321,573","4,845,044",1.356106,"2,106,255",True
7,2025,Orlando Arcia,30.0,SS,"4,314,297","796,919",0.957388,"1,650,520",True
8,2025,Chris Taylor,34.0,3B,"1,735,123","-980,696",0.858941,"5,411,096",False
9,2025,Tyler Wade,30.0,SS,"1,228,300","1,419,991",1.051937,"780,000",True


In [19]:
free_agency_2026_display.to_csv("Free Agent Batters 2026 Eval.csv", index=False)